In [1]:
import numpy as np 

import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim 

import requests 
from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    brotlipy-0.7.0             |py36h8c4c3a4_1000         346 KB  conda-forge
    chardet-3.0.4              |py36h9f0ad1d_1006         188 KB  conda-forge
    cryptography-2.8           |   py36h45558ae_2         628 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    pandas-1.0.3               |   py36h830a2c2_1        11.1 MB  conda-forge


In [2]:
df_lib = pd.read_csv('Library_Usage.csv')
df_lib = pd.DataFrame(df_lib, columns= ['Patron Type Definition','Total Checkouts', 'Home Library Definition'])

df_lib['Total Checkouts'] = df_lib['Total Checkouts'].astype('int')

df_lib.sort_values(['Total Checkouts'], ascending = False, inplace = True)
df_lib.head()

,Patron Type Definition,Total Checkouts,Home Library Definition
294771,ADULT,35907,Visitacion Valley
163107,SENIOR,25223,Main Library
147572,ADULT,24093,Main Library
199745,SENIOR,18397,Main Library
3204,SPECIAL,18064,Main Library


In [4]:
df_libpop = df_lib['Home Library Definition'].value_counts()
#
df_libpop.columns = ['Library Name', 'Patron Number']
#df_libpop =df_libpop, columns= ['Library Name','Patron Number'])


df_libpop.sort_values(0)

Branch Bookmobile (Marina)                 7
Branch Bookmobile (Excelsior)             23
Branch Bookmobile (Sunset)                27
Branch Bookmobile (West Portal)          104
Library on Wheels                        782
Children's Bookmobile                    807
Unknown                                 1498
Ocean View                              2914
Golden Gate Valley                      4381
Visitacion Valley                       6833
Anza                                    7183
Potrero                                 7196
Park                                    8271
Noe Valley/Sally Brunn                  8399
Bayview/Linda Brooks-Burton             8417
Presidio                                8652
Portola                                 8659
Eureka Valley/Harvey Milk Memorial      8708
North Beach                             9162
Bernal Heights                          9630
Western Addition                        9706
Parkside                                9744
Glen Park 

In [5]:
df_geo = pd.read_csv('Library Info - LatLong.csv')
df_geo.columns =['Neighborhood', 'Latitude', 'Longitude']
df_geo.head()

,Neighborhood,Latitude,Longitude
0,Bayview Hunters Point,37.7304,122.3844
1,Bernal Heights,37.7389,122.4152
2,Castro/Upper Market,37.7609,122.4350
3,Chinatown,37.7941,122.4078
4,Excelsior,37.7244,122.4272


In [6]:
CLIENT_ID = 'S20GHF0JBXEBSJ4P5HAZNJTHTMNUUIAM1LCGE2GRNSXECHCP' # your Foursquare ID
CLIENT_SECRET = 'NT2MECWVL5IW2FLNKL1RB30FNHAVKZUZU4Z23XQIXT24IG5C' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: S20GHF0JBXEBSJ4P5HAZNJTHTMNUUIAM1LCGE2GRNSXECHCP
CLIENT_SECRET:NT2MECWVL5IW2FLNKL1RB30FNHAVKZUZU4Z23XQIXT24IG5C


In [7]:
address = 'San Francisco'

geolocator = Nominatim(user_agent="SF_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of San Francisco are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of San Francisco are 37.7790262, -122.4199061.


In [8]:
map_SF = folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, lng, neighborhood in zip(df_geo['Latitude'], df_geo['Longitude'], df_geo['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_SF)  
    
map_SF

In [10]:
radius = 15000
limit = 100
search_query = 'Library'
LATITUDE = 37.7790262
LONGITUDE = -122.4199061
url = 'https://api.foursquare.com/v2/venues/search?&client_id=S20GHF0JBXEBSJ4P5HAZNJTHTMNUUIAM1LCGE2GRNSXECHCP&client_secret=NT2MECWVL5IW2FLNKL1RB30FNHAVKZUZU4Z23XQIXT24IG5C&v=20180605&ll=37.7790262,-122.4199061&query=Library&radius=15000&limit=100'.format(CLIENT_ID, CLIENT_SECRET, VERSION, LATITUDE, LONGITUDE, search_query, radius, limit)
url

'https://api.foursquare.com/v2/venues/search?&client_id=S20GHF0JBXEBSJ4P5HAZNJTHTMNUUIAM1LCGE2GRNSXECHCP&client_secret=NT2MECWVL5IW2FLNKL1RB30FNHAVKZUZU4Z23XQIXT24IG5C&v=20180605&ll=37.7790262,-122.4199061&query=Library&radius=15000&limit=100'

In [11]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e9a55d76d8c56001bcdcd14'},
 'response': {'venues': [{'id': '43f5f8a9f964a520a92f1fe3',
    'name': 'San Francisco Public Library',
    'location': {'address': '100 Larkin St',
     'crossStreet': 'at Grove St',
     'lat': 37.77881442156546,
     'lng': -122.41551743910111,
     'labeledLatLngs': [{'label': 'display',
       'lat': 37.77881442156546,
       'lng': -122.41551743910111}],
     'distance': 386,
     'postalCode': '94102',
     'cc': 'US',
     'neighborhood': 'Civic Center',
     'city': 'San Francisco',
     'state': 'CA',
     'country': 'United States',
     'formattedAddress': ['100 Larkin St (at Grove St)',
      'San Francisco, CA 94102',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d12f941735',
      'name': 'Library',
      'pluralName': 'Libraries',
      'shortName': 'Library',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/library_',
       'suffix': '.png'},
      'primary': 

In [12]:
venues = pd.json_normalize(results['response']['venues'])
venues.columns

Index(['id', 'name', 'categories', 'referralId', 'hasPerk', 'location.address',
       'location.crossStreet', 'location.lat', 'location.lng',
       'location.labeledLatLngs', 'location.distance', 'location.postalCode',
       'location.cc', 'location.neighborhood', 'location.city',
       'location.state', 'location.country', 'location.formattedAddress',
       'venuePage.id'],
      dtype='object')

In [27]:
libraries = venues.filter(['name', 'categories', 'location.lat', 'location.lng'], axis=1) 
libraries.columns =['name', 'category', 'latitude', 'longitude']
libraries

,name,category,latitude,longitude
0,San Francisco Public Library,"[{'id': '4bf58dd8d48988d12f941735', 'name': 'L...",37.778814,-122.415517
1,Library for the Blind and Print Disabled,"[{'id': '4bf58dd8d48988d12f941735', 'name': 'L...",37.779076,-122.415724
2,UC Hastings College of the Law: Law Library,"[{'id': '4bf58dd8d48988d1a7941735', 'name': 'C...",37.780716,-122.415451
3,United States District Court Law Library,"[{'id': '4bf58dd8d48988d12f941735', 'name': 'L...",37.781800,-122.418661
4,San Francisco Conservatory of Music Library,"[{'id': '4bf58dd8d48988d1a7941735', 'name': 'C...",37.775494,-122.420281
5,San Francisco Public Library Collections & Tec...,"[{'id': '4bf58dd8d48988d124941735', 'name': 'O...",37.775029,-122.413675
6,Park Branch Library,"[{'id': '4bf58dd8d48988d12f941735', 'name': 'L...",37.770635,-122.450747
7,Western Addition Branch Library,"[{'id': '4bf58dd8d48988d12f941735', 'name': 'L...",37.784160,-122.437711
8,Eureka Valley/Harvey Milk Memorial Branch Library,"[{'id': '4bf58dd8d48988d12f941735', 'name': 'L...",37.764219,-122.432280
9,The Library,"[{'id': '4bf58dd8d48988d12f941735', 'name': 'L...",37.775338,-122.399886


In [29]:
libraries.drop([47,48,45,44,40,39,38,37,34,31,30,29,28,27,26,23,22,21,19,16,10,9,5,4,3,2,1], axis=0, inplace = True)

In [30]:
map = folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, lng, label in zip(libraries['latitude'], libraries['longitude'], libraries['name']):
        folium.features.CircleMarker(
            [lat, lng],
            radius=5,
            poup=label,
            fill=True,
            color='blue',
            fill_color='blue',
            fill_opacity=0.6
        ).add_to(map)
        
map